In [181]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
import torch
from collections import deque
import random
from math import sqrt

In [182]:
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255,0,0)
GREEN = (0,255,0)
BLUE = (0,0,255)

MAP_POINTS = [(0,100), (50,300), (10,400),(60,500), (200,560),
              (400,500),(500,400),(600,350),(700,325),
              (800,315),(900,800),(1100,800),(1000,600),
              (950,500),(975,400),(850,150),(500,325),
              (350,300),(120,350),(100,0)
             ]

In [187]:
#helpers
def slope(p1, p2) :
    return (p2[1] - p1[1]) * 1. / ((p2[0] - p1[0])+1e-5)
   
def y_intercept(slope, p1) :
    return p1[1] - 1. * slope * p1[0]
   
def intersecta(line1, line2) :
    min_allowed = 1e-5
    big_value = 1e10
    m1 = slope(line1[0], line1[1])
    b1 = y_intercept(m1, line1[0])
    m2 = slope(line2[0], line2[1])
    b2 = y_intercept(m2, line2[0])
    if abs(m1 - m2) < min_allowed :
        x = big_value
    else :
        x = (b2 - b1) / (m1 - m2)
    y = m1 * x + b1
    y2 = m2 * x + b2
    return (int(x),int(y))

def segment_intersect(line1, line2) :
    intersection_pt = intersecta(line1, line2)
    
    if (line1[0][0] < line1[1][0]) :
        if intersection_pt[0] < line1[0][0] or intersection_pt[0] > line1[1][0] :
            return None
    else :
        if intersection_pt[0] > line1[0][0] or intersection_pt[0] < line1[1][0] :
            return None
         
    if (line2[0][0] < line2[1][0]) :
        if intersection_pt[0] < line2[0][0] or intersection_pt[0] > line2[1][0] :
            return None
    else :
        if intersection_pt[0] > line2[0][0] or intersection_pt[0] < line2[1][0] :
            return None

    return intersection_pt

In [188]:
class Car():
    def __init__(self):
        self.center_pos = pygame.math.Vector2(50, 50)
        self.corners = [
            pygame.math.Vector2(-10,-20),
            pygame.math.Vector2(10,-20),
            pygame.math.Vector2(10,20),
            pygame.math.Vector2(-10,20)
        ]
        self.corners_pos = [x+self.center_pos for x in self.corners]
        self.sensors = [
            pygame.math.Vector2(0,1000),
            pygame.math.Vector2(966,258),
            pygame.math.Vector2(-966,258)
        ]
        self.sensors_pos = [x+self.center_pos for x in self.sensors]
        self.velocity = pygame.math.Vector2(0,10)
        self.angle = 0
        
    def update(self):
        self.angle += random.randint(-1,1) * 10
        self.angle = self.angle % 360
        self.center_pos = self.center_pos + self.velocity.rotate(self.angle)
        self.corners_pos = [x.rotate(self.angle)+self.center_pos for x in self.corners]
        self.sensors_pos = [x.rotate(self.angle)+self.center_pos for x in self.sensors]
        
    def reset(self):
        self.__init__()
        

In [189]:
pygame.init()
 
screen = pygame.display.set_mode([1000, 700])
 
pygame.display.set_caption('Comp 432 custom car env')

car = Car()
 
clock = pygame.time.Clock()
done = False

while not done:
 
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
            
    car.update()
    
    #generate signals
    intersects = []
    signals = []
    for sensor in car.sensors_pos:
        sensor_intersects = []
        for i in range(len(MAP_POINTS)-1):
            intersect = segment_intersect([MAP_POINTS[i],MAP_POINTS[i+1]],[(car.center_pos.x,car.center_pos.y),(sensor.x,sensor.y)])
            if(intersect != None):
                sensor_intersects.append(intersect)
                
        if len(sensor_intersects) == 0:
            intersects.append((-100,-100))
            signals.append(1000)
        else:
            sensor_distances = [sqrt((x[0] - car.center_pos.x)**2 +(x[1] - car.center_pos.y)**2) for x in sensor_intersects]
            intersects.append(sensor_intersects[sensor_distances.index(min(sensor_distances))])
            signals.append(min(sensor_distances))

    #check car colision
    for signal in signals:
        if signal < 15:
            car.reset()

    # -- Draw everything
    # Clear screen
    screen.fill(BLACK)
    
    #draw map
    pygame.draw.lines(screen,WHITE,False,MAP_POINTS)
    
    #draw goal
    pygame.draw.circle(screen,GREEN,(950,650),20)
    
    #draw car
    pygame.draw.lines(screen,BLUE,True,[(vect.x,vect.y) for vect in car.corners_pos])
    
    
    #draw sensors
    for sensor in car.sensors_pos:
        pygame.draw.line(screen,GREEN,(car.center_pos.x,car.center_pos.y),(sensor.x,sensor.y))
        
    for intersect in intersects:
        pygame.draw.circle(screen,RED,intersect,5)

    pygame.display.flip()
    clock.tick(1)
    
pygame.quit()